In [31]:
import torch
import torch.nn.functional as F

torch.manual_seed(1)

# What We Want to Implement
$$ cost(W) = -\frac{1}{n}\sum^n_{i=1}\sum^k_{j=1}y_j^{(i)}log(p_j^{(i)}) $$

## Low-Level Implementation

In [32]:
z = torch.FloatTensor([1, 2, 3])

hypotheses = F.softmax(z, dim=0)
print(hypotheses)  # Softmax 결과 출력
print(hypotheses.sum())  # Softmax 결과의 합 출력 (항상 1이어야 함)

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


In [33]:
z = torch.rand(3, 5, requires_grad=True)
print(z)

hypotheses = F.softmax(z, dim=1) #dim=1의 의미 : 행 단위로 softmax 적용
print(hypotheses)

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)
tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [34]:
y = torch.randint(5, (3,)).long()
print(y)
print(y.unsqueeze(1))#1차원 텐서를 2차원 텐서로 변환

tensor([0, 2, 1])
tensor([[0],
        [2],
        [1]])


In [35]:
y_one_hot = torch.zeros_like(hypotheses)
print(y_one_hot)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) #dim=1, index=y.unsqueeze(1), value=1
print(y_one_hot)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [36]:
cost = (-y_one_hot * torch.log(hypotheses)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


## High-Level Implementation

In [37]:
print(torch.log(F.softmax(z, dim=1)))
print(F.log_softmax(z, dim=1))

(-y_one_hot * F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)
tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)


tensor(1.4689, grad_fn=<MeanBackward0>)

In [ ]:
F.nll_loss(F.log_softmax(z, dim=1), y)  #nll_loss에는 One-Hot Encoding이 필요 없음

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [39]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)